# ClimAgent quick start

In [4]:
from climagent.agent.climagent import ClimAgent
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
import os
from dotenv import load_dotenv
import xarray as xr

load_dotenv("../credentials.env")

import warnings
warnings.filterwarnings("ignore")

In [5]:
# Initialize LLM
llm = AzureChatOpenAI(
    deployment_name=os.getenv("GPT_DEPLOYMENT_NAME"),
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_ENDPOINT"),
    temperature=0,
)

In [6]:
dataset_path = "../example_datasets/cckp/store/CMIP6_timeseries.zarr"
dataset = xr.open_zarr(dataset_path, decode_timedelta=True)
dataset

<xarray.Dataset> Size: 2GB
Dimensions:     (percentile: 3, scenario: 3, time: 86, lat: 121, lon: 121)
Coordinates:
  * lat         (lat) float64 968B 30.0 30.25 30.5 30.75 ... 59.5 59.75 60.0
  * lon         (lon) float64 968B 0.0 0.25 0.5 0.75 ... 29.25 29.5 29.75 30.0
  * percentile  (percentile) <U6 72B 'median' 'p10' 'p90'
  * scenario    (scenario) <U6 72B 'ssp126' 'ssp245' 'ssp585'
  * time        (time) datetime64[ns] 688B 2015-07-01 2016-07-01 ... 2100-07-01
Data variables: (12/31)
    cdd65       (percentile, scenario, time, lat, lon) float32 45MB dask.array<chunksize=(3, 3, 86, 121, 121), meta=np.ndarray>
    csdi        (percentile, scenario, time, lat, lon) float64 91MB dask.array<chunksize=(3, 3, 86, 121, 121), meta=np.ndarray>
    fd          (percentile, scenario, time, lat, lon) float64 91MB dask.array<chunksize=(3, 3, 86, 121, 121), meta=np.ndarray>
    hd30        (percentile, scenario, time, lat, lon) float64 91MB dask.array<chunksize=(3, 3, 86, 121, 121), meta=np.ndarray>
    hd35        (percentile, scenario, time, lat, lon) float64 91MB dask.array<chunksize=(3, 3, 86, 121, 121), meta=np.ndarray>
    hd40        (percentile, scenario, time, lat, lon) float64 91MB dask.array<chunksize=(3, 3, 86, 121, 121), meta=np.ndarray>
    ...          ...
    tr23        (percentile, scenario, time, lat, lon) float64 91MB dask.array<chunksize=(3, 3, 86, 121, 121), meta=np.ndarray>
    tr26        (percentile, scenario, time, lat, lon) float64 91MB dask.array<chunksize=(3, 3, 86, 121, 121), meta=np.ndarray>
    tr29        (percentile, scenario, time, lat, lon) float64 91MB dask.array<chunksize=(3, 3, 86, 121, 121), meta=np.ndarray>
    tr32        (percentile, scenario, time, lat, lon) float64 91MB dask.array<chunksize=(3, 3, 86, 121, 121), meta=np.ndarray>
    txx         (percentile, scenario, time, lat, lon) float64 91MB dask.array<chunksize=(3, 3, 86, 121, 121), meta=np.ndarray>
    wsdi        (percentile, scenario, time, lat, lon) timedelta64[ns] 91MB dask.array<chunksize=(3, 3, 86, 121, 121), meta=np.ndarray>
Attributes: (12/43)
    Conventions:           CF-1.7 CMIP-6.2
    activity_id:           ScenarioMIP
    climate_data_id:       CMIP6
    cmip6-ng:              Some daily data has been acquired through the CMIP...
    cmor_version:          3.4.0
    creation_date:         2021-11-05T01:00:00Z
    ...                    ...
    wb_scenario_label:     ssp126
    wb_statistic_label:    mean
    wb_truncation_label:   2
    wb_type_label:         timeseries
    wb_variable_label:     tas
    wb_variant_label:      all

In [4]:
agent = ClimAgent(dataset, llm)
# agent.graph

In [5]:
query: str = (
    "I want to assess whether Rome is expected to experience heat waves conditions hazardous to the population in the period 2040-2060.\n"
    # "Proceed as follows:\n"
    # "1. Select the data for Rome.\n"
    # "2. Select the data for the period 2040-2060.\n"
    # "3. Select the data for the variables that you think are most relevant to assess the heat waves conditions.\n"
    # "4. Aggregate percentiles using mean. "
    # "5. Do not aggregate over the scenario dimension.\n"
    # "6. Describe the results you obtain and answer the question.\n"
)

messages = [HumanMessage(query)]
response = agent.run(messages)

In [6]:
print(response['messages'][-1].content)

The analysis of heat wave conditions in Rome for the period 2040-2060 reveals the following insights based on the selected variables:

1. **Maximum Temperature (tasmax)**:
   - Projected maximum temperatures for different scenarios are:
     - SSP126: 22.01°C
     - SSP245: 22.19°C
     - SSP585: 22.78°C

2. **Heat Days (hd30)**:
   - The number of days with maximum temperatures exceeding 30°C:
     - SSP126: 73.59 days
     - SSP245: 76.75 days
     - SSP585: 86.10 days

3. **Heat Days (hd35)**:
   - The number of days with maximum temperatures exceeding 35°C:
     - SSP126: 11.17 days
     - SSP245: 12.65 days
     - SSP585: 22.86 days

4. **Heat Index (hi)**:
   - The average heat index values for different scenarios:
     - SSP126: 16.94
     - SSP245: 17.21
     - SSP585: 17.76

### Conclusion:
Based on the projections, Rome is expected to experience an increase in the number of heat days, particularly under the SSP585 scenario, which indicates a significant rise in hazardous heat

In [7]:
agent.dataset_state.history

['Subset on lat(41.9028)',
 'Subset on lon(12.4964)',
 'Subset on time(2040-01-01:2060-12-31)',
 "Selected variables:['tasmax', 'hd30', 'hd35', 'hi'])",
 "Aggregated on ['time'] using mean",
 "Aggregated on ['percentile'] using mean"]

In [8]:
agent.dataset_state.dataset

<xarray.Dataset> Size: 160B
Dimensions:   (scenario: 3)
Coordinates:
    lat       float64 8B 42.0
    lon       float64 8B 12.5
  * scenario  (scenario) <U6 72B 'ssp126' 'ssp245' 'ssp585'
Data variables:
    tasmax    (scenario) float32 12B dask.array<chunksize=(3,), meta=np.ndarray>
    hd30      (scenario) float64 24B dask.array<chunksize=(3,), meta=np.ndarray>
    hd35      (scenario) float64 24B dask.array<chunksize=(3,), meta=np.ndarray>
    hi        (scenario) float32 12B dask.array<chunksize=(3,), meta=np.ndarray>